# Microplastics Classification Method Comparison

**Comparing 3 Deep Learning Classification Methods**

This notebook trains and compares 3 different classification architectures for identifying microplastic particle types.

## Task: Multi-class Classification
- **Input**: RGB images of microplastic particles
- **Output**: Class label (Fragment, Bead, Fiber)
- **Dataset**: micro-plastics-sem from datasets_subset_Exp2
- **Framework**: PyTorch with transfer learning

## Classification Methods Compared

### 1. **ResNet50** (Residual Networks)
- Deep architecture (50 layers) with skip connections
- Pre-trained on ImageNet → transfer learning
- Strong baseline for image classification
- Good at capturing complex features

### 2. **EfficientNet-B0** (Efficient Architecture)
- Optimized compound scaling of depth, width, resolution
- Fewer parameters, faster inference
- State-of-the-art efficiency
- Better accuracy-to-parameter ratio

### 3. **MobileNetV2** (Lightweight Mobile Architecture)
- Designed for mobile/edge devices
- Inverted residual blocks with linear bottlenecks
- Very fast inference, small model size
- Great for deployment scenarios

## Classification Categories

- **Fragment**: Irregular hard plastic pieces
- **Bead**: Spherical plastic pellets
- **Fiber**: Thread-like plastic strands

## Setup Instructions for Kaggle

1. Upload `micro-plastics-sem` dataset to Kaggle
2. Enable GPU: Settings → Accelerator → GPU T4 x2
3. Add dataset: + Add data → micro-plastics-sem
4. Run all cells

**Expected Runtime**: ~30-40 minutes on Kaggle GPU (20 epochs per model)

## 1. Setup and Installation

In [ ]:
# Install required packages with compatible versions
# IMPORTANT: Install in this specific order to avoid conflicts

# Step 1: Uninstall problematic packages
!pip uninstall -y numpy scipy -q

# Step 2: Install NumPy 1.26.x (compatible with scipy and mkl packages)
!pip install -q "numpy==1.26.4"

# Step 3: Install scipy with the compatible NumPy
!pip install -q "scipy>=1.11.0,<2.0.0"

# Step 4: Install other packages
!pip install -q albumentations opencv-python-headless scikit-learn matplotlib seaborn tqdm pandas timm

import torch
import torch.nn as nn
import torch.optim as optim
import torchvision.models as models
from torch.utils.data import Dataset, DataLoader, random_split

import albumentations as A
from albumentations.pytorch import ToTensorV2

import cv2
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os
import glob
import time
from tqdm import tqdm
import warnings
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, classification_report
)
from collections import Counter, defaultdict
import timm

warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
torch.manual_seed(42)
np.random.seed(42)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(42)

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f'Using device: {device}')
if torch.cuda.is_available():
    print(f'GPU: {torch.cuda.get_device_name(0)}')
    print(f'CUDA version: {torch.version.cuda}')
    print(f'Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.2f} GB')

print(f'\nPyTorch version: {torch.__version__}')
print(f'NumPy version: {np.__version__}')

## 2. Dataset Configuration

The dataset should be organized with class folders:
```
/kaggle/input/micro-plastics-sem/Microplastics_SEM/dataset1/dataset1/datasets_subset_Exp2/
├── bead/
│   ├── img001.jpg
│   └── ...
├── fragment/
└── fibre_1/
```

In [ ]:
# Dataset configuration
print("Searching for dataset...")
print("="*70)

# Define class names (matching folder names in datasets_subset_Exp2)
CLASS_NAMES = ['bead', 'fragment', 'fibre_1']  # Will map fibre_1 to 'fiber' for display
DISPLAY_NAMES = ['Bead', 'Fragment', 'Fiber']  # User-friendly names
NUM_CLASSES = len(CLASS_NAMES)

# Auto-detect dataset path
DATASET_PATH = None

# Check Kaggle paths
if os.path.exists('/kaggle/input/'):
    print("Running on Kaggle")
    
    # Try common paths
    possible_paths = [
        '/kaggle/input/microplastics-sem-dataset/Microplastics_SEM/dataset1/dataset1/datasets_subset_Exp2'
    ]
    
    for path in possible_paths:
        if os.path.exists(path):
            DATASET_PATH = path
            print(f"Found dataset at: {DATASET_PATH}")
            break
    
    if not DATASET_PATH:
        # List available datasets
        available = os.listdir('/kaggle/input/')
        print(f"\nAvailable datasets:")
        for ds in available:
            print(f"  - {ds}")
else:
    # Local path
    local_path = r"C:\Users\SHIVAPREETHAM ROHITH\Desktop\AI\micro-plastic-detection\data\Microplastics_SEM\dataset1\dataset1\datasets_subset_Exp2"
    if os.path.exists(local_path):
        DATASET_PATH = local_path
        print(f"Using local dataset: {DATASET_PATH}")

# Create output directories
output_dir = '/kaggle/working/' if os.path.exists('/kaggle/working/') else './output/'
os.makedirs(os.path.join(output_dir, 'models'), exist_ok=True)
os.makedirs(os.path.join(output_dir, 'results'), exist_ok=True)

# Verify dataset structure
if DATASET_PATH:
    print("\n" + "="*70)
    print("DATASET STRUCTURE")
    print("="*70)
    
    class_counts = {}
    for class_name in CLASS_NAMES:
        class_path = os.path.join(DATASET_PATH, class_name)
        if os.path.exists(class_path):
            images = []
            for ext in ['*.jpg', '*.jpeg', '*.png', '*.tif', '*.JPG', '*.JPEG', '*.PNG', '*.TIF']:
                images.extend(glob.glob(os.path.join(class_path, ext)))
            class_counts[class_name] = len(images)
            display_name = DISPLAY_NAMES[CLASS_NAMES.index(class_name)]
            print(f"  {display_name:12s}: {len(images):5d} images")
        else:
            class_counts[class_name] = 0
            print(f"  {class_name:12s}: NOT FOUND")
    
    total_images = sum(class_counts.values())
    print(f"\n  Total images: {total_images}")
    print("="*70)
    
    if total_images == 0:
        print("\n⚠️ WARNING: No images found in class folders!")
        print(f"Path: {DATASET_PATH}")
        print("Please verify the dataset structure.")
        raise FileNotFoundError("No images found in dataset")
else:
    print("\n⚠️ WARNING: Dataset not found!")
    print("Please add 'microplastics-sem-dataset' dataset in Kaggle:")
    print("  + Add data → microplastics-sem-dataset")
    raise FileNotFoundError("Dataset not found")

## 3. Dataset Class and Data Loading

In [ ]:
class MicroplasticsClassificationDataset(Dataset):
    """Dataset for microplastics classification"""
    
    def __init__(self, data_dir, class_names, transform=None, image_size=(224, 224)):
        self.data_dir = data_dir
        self.class_names = class_names
        self.transform = transform
        self.image_size = image_size
        
        # Build file list
        self.images = []
        self.labels = []
        
        for class_idx, class_name in enumerate(class_names):
            class_dir = os.path.join(data_dir, class_name)
            if os.path.exists(class_dir):
                for ext in ['*.jpg', '*.jpeg', '*.png', '*.tif', '*.JPG', '*.JPEG', '*.PNG', '*.TIF']:
                    files = glob.glob(os.path.join(class_dir, ext))
                    self.images.extend(files)
                    self.labels.extend([class_idx] * len(files))
        
        print(f"Loaded {len(self.images)} images from {data_dir}")
    
    def __len__(self):
        return len(self.images)
    
    def __getitem__(self, idx):
        img_path = self.images[idx]
        label = self.labels[idx]
        
        # Load image
        image = cv2.imread(img_path)
        if image is None:
            raise ValueError(f"Failed to load image: {img_path}")
        
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, self.image_size)
        
        # Apply transforms
        if self.transform:
            augmented = self.transform(image=image)
            image = augmented['image']
        
        return image, label

# Data augmentation for training
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.VerticalFlip(p=0.5),
    A.Rotate(limit=90, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
    A.OneOf([
        A.MotionBlur(blur_limit=5, p=1.0),
        A.MedianBlur(blur_limit=5, p=1.0),
        A.GaussianBlur(blur_limit=5, p=1.0),
    ], p=0.3),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.2, rotate_limit=45, p=0.5),
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

val_transform = A.Compose([
    A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
    ToTensorV2()
])

# Create dataset
if DATASET_PATH:
    print("\nCreating dataset...")
    full_dataset = MicroplasticsClassificationDataset(
        DATASET_PATH,
        CLASS_NAMES,
        transform=train_transform,
        image_size=(224, 224)
    )
    
    # Split dataset: 70% train, 20% val, 10% test
    train_size = int(0.7 * len(full_dataset))
    val_size = int(0.2 * len(full_dataset))
    test_size = len(full_dataset) - train_size - val_size
    
    train_dataset, val_dataset, test_dataset = random_split(
        full_dataset,
        [train_size, val_size, test_size],
        generator=torch.Generator().manual_seed(42)
    )
    
    # Apply validation transform to val and test
    val_dataset.dataset.transform = val_transform
    test_dataset.dataset.transform = val_transform
    
    print(f"\nDataset split:")
    print(f"  Train: {len(train_dataset)} ({len(train_dataset)/len(full_dataset)*100:.1f}%)")
    print(f"  Val:   {len(val_dataset)} ({len(val_dataset)/len(full_dataset)*100:.1f}%)")
    print(f"  Test:  {len(test_dataset)} ({len(test_dataset)/len(full_dataset)*100:.1f}%)")
    
    # Create data loaders
    BATCH_SIZE = 32
    NUM_WORKERS = 2
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    val_loader = DataLoader(
        val_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=True if torch.cuda.is_available() else False
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS
    )
    
    print(f"\nData loaders created:")
    print(f"  Batch size: {BATCH_SIZE}")
    print(f"  Train batches: {len(train_loader)}")
    print(f"  Val batches:   {len(val_loader)}")
    print(f"  Test batches:  {len(test_loader)}")

## 4. Visualize Sample Data

In [ ]:
def visualize_samples(dataloader, class_names, num_samples=8):
    """Visualize sample images from each class"""
    fig, axes = plt.subplots(2, 4, figsize=(20, 10))
    axes = axes.flatten()
    
    data_iter = iter(dataloader)
    images, labels = next(data_iter)
    
    for i in range(min(num_samples, len(images))):
        # Denormalize image
        img = images[i].permute(1, 2, 0).numpy()
        img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
        img = np.clip(img, 0, 1)
        
        label = labels[i].item()
        class_name = class_names[label]
        
        axes[i].imshow(img)
        axes[i].set_title(f'Class: {class_name}', fontsize=14, fontweight='bold')
        axes[i].axis('off')
    
    plt.tight_layout()
    save_path = os.path.join(output_dir, 'results', 'sample_images.png')
    plt.savefig(save_path, dpi=150, bbox_inches='tight')
    plt.show()
    
    # Show class distribution
    label_counts = Counter(labels.numpy())
    print(f"\nClass distribution in batch:")
    for class_idx, count in sorted(label_counts.items()):
        print(f"  {class_names[class_idx]:12s}: {count} samples")

if DATASET_PATH:
    print("Visualizing sample data...")
    visualize_samples(train_loader, CLASS_NAMES)

## 5. Model Architectures

We'll compare 3 classification architectures:
1. **ResNet50** (50 layers, strong baseline)
2. **EfficientNet-B0** (efficient scaling, best accuracy/params)
3. **MobileNetV2** (lightweight, fast inference)

In [ ]:
def create_models(num_classes=3):
    """Create all 3 classification models"""
    models_dict = {}
    
    # Model 1: ResNet50
    resnet50 = models.resnet50(pretrained=True)
    num_features = resnet50.fc.in_features
    resnet50.fc = nn.Sequential(
        nn.Dropout(0.5),
        nn.Linear(num_features, 512),
        nn.ReLU(),
        nn.Dropout(0.3),
        nn.Linear(512, num_classes)
    )
    models_dict['ResNet50'] = resnet50
    
    # Model 2: EfficientNet-B0
    efficientnet = timm.create_model('efficientnet_b0', pretrained=True)
    num_features = efficientnet.classifier.in_features
    efficientnet.classifier = nn.Sequential(
        nn.Dropout(0.4),
        nn.Linear(num_features, 256),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(256, num_classes)
    )
    models_dict['EfficientNet-B0'] = efficientnet
    
    # Model 3: MobileNetV2
    mobilenet = models.mobilenet_v2(pretrained=True)
    num_features = mobilenet.classifier[1].in_features
    mobilenet.classifier = nn.Sequential(
        nn.Dropout(0.3),
        nn.Linear(num_features, 256),
        nn.ReLU(),
        nn.Dropout(0.2),
        nn.Linear(256, num_classes)
    )
    models_dict['MobileNetV2'] = mobilenet
    
    return models_dict

print("Creating models...")
all_models = create_models(NUM_CLASSES)

print("\n" + "="*70)
print("MODEL ARCHITECTURES")
print("="*70)
for name, model in all_models.items():
    total_params = sum(p.numel() for p in model.parameters())
    trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print(f"{name:20s}: {total_params:>12,} params ({trainable_params:>12,} trainable)")
print("="*70)

## 6. Training Functions

In [ ]:
def train_one_model(model_name, model, train_loader, val_loader, num_epochs=20, lr=1e-4):
    """
    Train a single classification model
    
    Args:
        model_name: Name of the model
        model: PyTorch model
        train_loader: Training data loader
        val_loader: Validation data loader
        num_epochs: Number of training epochs
        lr: Learning rate
    
    Returns:
        Dictionary with training history and metrics
    """
    print(f"\n{'='*70}")
    print(f"Training {model_name}")
    print(f"{'='*70}")
    
    model = model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode='max', factor=0.5, patience=3, verbose=False
    )
    
    history = {
        'train_loss': [],
        'val_loss': [],
        'train_acc': [],
        'val_acc': [],
        'val_precision': [],
        'val_recall': [],
        'val_f1': [],
    }
    
    best_val_acc = 0.0
    best_epoch = 0
    start_time = time.time()
    
    for epoch in range(num_epochs):
        # Training phase
        model.train()
        train_loss = 0.0
        train_preds = []
        train_labels = []
        
        train_bar = tqdm(train_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Train]", leave=False)
        for images, labels in train_bar:
            images, labels = images.to(device), labels.to(device)
            
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            
            train_loss += loss.item()
            _, preds = torch.max(outputs, 1)
            train_preds.extend(preds.cpu().numpy())
            train_labels.extend(labels.cpu().numpy())
            
            train_bar.set_postfix({'loss': loss.item()})
        
        train_loss /= len(train_loader)
        train_acc = accuracy_score(train_labels, train_preds)
        
        # Validation phase
        model.eval()
        val_loss = 0.0
        val_preds = []
        val_labels = []
        
        with torch.no_grad():
            val_bar = tqdm(val_loader, desc=f"Epoch {epoch+1}/{num_epochs} [Val]  ", leave=False)
            for images, labels in val_bar:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                
                val_loss += loss.item()
                _, preds = torch.max(outputs, 1)
                val_preds.extend(preds.cpu().numpy())
                val_labels.extend(labels.cpu().numpy())
                
                val_bar.set_postfix({'loss': loss.item()})
        
        val_loss /= len(val_loader)
        val_acc = accuracy_score(val_labels, val_preds)
        val_prec = precision_score(val_labels, val_preds, average='weighted', zero_division=0)
        val_rec = recall_score(val_labels, val_preds, average='weighted', zero_division=0)
        val_f1 = f1_score(val_labels, val_preds, average='weighted', zero_division=0)
        
        # Update learning rate
        scheduler.step(val_acc)
        
        # Save history
        history['train_loss'].append(train_loss)
        history['val_loss'].append(val_loss)
        history['train_acc'].append(train_acc)
        history['val_acc'].append(val_acc)
        history['val_precision'].append(val_prec)
        history['val_recall'].append(val_rec)
        history['val_f1'].append(val_f1)
        
        # Print progress
        if (epoch + 1) % 5 == 0 or epoch == 0:
            print(f"Epoch {epoch+1:2d}/{num_epochs} | "
                  f"Loss: {train_loss:.4f}/{val_loss:.4f} | "
                  f"Acc: {train_acc:.4f}/{val_acc:.4f} | "
                  f"F1: {val_f1:.4f}")
        
        # Save best model
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            best_epoch = epoch + 1
            model_path = os.path.join(output_dir, 'models', f'{model_name.lower().replace("-", "_")}_best.pth')
            torch.save(model.state_dict(), model_path)
    
    training_time = time.time() - start_time
    
    print(f"\n✓ {model_name} training complete!")
    print(f"  Best Val Acc: {best_val_acc:.4f} (epoch {best_epoch})")
    print(f"  Training time: {training_time/60:.1f} minutes")
    
    return {
        'model': model,
        'history': history,
        'best_val_acc': best_val_acc,
        'best_epoch': best_epoch,
        'training_time': training_time
    }

## 7. Train All Models

In [ ]:
# Training configuration
NUM_EPOCHS = 20
LEARNING_RATE = 1e-4

print("="*70)
print("TRAINING CLASSIFICATION MODELS")
print("="*70)
print(f"Dataset: {DATASET_PATH}")
print(f"Classes: {NUM_CLASSES} ({', '.join(DISPLAY_NAMES)})")
print(f"Epochs: {NUM_EPOCHS}")
print(f"Learning rate: {LEARNING_RATE}")
print(f"Device: {device}")
print("="*70)

# Train each model
results = {}
for model_name, model in all_models.items():
    results[model_name] = train_one_model(
        model_name, model, train_loader, val_loader,
        num_epochs=NUM_EPOCHS, lr=LEARNING_RATE
    )

print("\n" + "="*70)
print("ALL MODELS TRAINED")
print("="*70)

## 8. Comparison Results

In [ ]:
# Create comparison table
comparison_data = []
for model_name, result in results.items():
    model = result['model']
    total_params = sum(p.numel() for p in model.parameters())
    
    comparison_data.append({
        'Model': model_name,
        'Best Val Acc': f"{result['best_val_acc']:.4f}",
        'Best Epoch': result['best_epoch'],
        'Final Val Acc': f"{result['history']['val_acc'][-1]:.4f}",
        'Final Val F1': f"{result['history']['val_f1'][-1]:.4f}",
        'Parameters (M)': f"{total_params/1e6:.2f}",
        'Training Time (min)': f"{result['training_time']/60:.1f}"
    })

comparison_df = pd.DataFrame(comparison_data)
comparison_df = comparison_df.sort_values('Best Val Acc', ascending=False)

print("\n" + "="*70)
print("MODEL COMPARISON")
print("="*70)
print(comparison_df.to_string(index=False))
print("="*70)

# Save comparison table
comparison_df.to_csv(os.path.join(output_dir, 'results', 'classification_comparison.csv'), index=False)
print("\nComparison table saved!")

## 9. Training Curves Comparison

In [ ]:
# Plot training curves for all models
fig, axes = plt.subplots(2, 3, figsize=(22, 12))

colors = {'ResNet50': '#E74C3C', 'EfficientNet-B0': '#3498DB', 'MobileNetV2': '#2ECC71'}
metrics = [
    ('train_loss', 'Training Loss', axes[0, 0]),
    ('val_loss', 'Validation Loss', axes[0, 1]),
    ('val_acc', 'Validation Accuracy', axes[0, 2]),
    ('val_precision', 'Validation Precision', axes[1, 0]),
    ('val_recall', 'Validation Recall', axes[1, 1]),
    ('val_f1', 'Validation F1 Score', axes[1, 2]),
]

for metric_key, title, ax in metrics:
    for model_name, result in results.items():
        values = result['history'][metric_key]
        ax.plot(range(1, len(values)+1), values,
               label=model_name, linewidth=2.5, color=colors[model_name])
    
    ax.set_title(title, fontsize=14, fontweight='bold')
    ax.set_xlabel('Epoch', fontsize=11)
    ax.set_ylabel(title.split()[-1], fontsize=11)
    ax.legend(fontsize=10)
    ax.grid(True, alpha=0.3, linestyle='--')

plt.suptitle('Microplastics Classification - Training Comparison',
             fontsize=16, fontweight='bold', y=0.995)
plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'results', 'training_curves_comparison.png'),
           dpi=150, bbox_inches='tight')
plt.show()

print("Training curves saved!")

## 10. Test Set Evaluation (Best Model)

In [ ]:
# Find best model
best_model_name = max(results.items(), key=lambda x: x[1]['best_val_acc'])[0]
best_model = results[best_model_name]['model']

print(f"\nBest model: {best_model_name}")
print(f"Best validation accuracy: {results[best_model_name]['best_val_acc']:.4f}")

# Load best checkpoint
checkpoint_path = os.path.join(output_dir, 'models', f'{best_model_name.lower().replace("-", "_")}_best.pth')
best_model.load_state_dict(torch.load(checkpoint_path))
best_model.eval()

# Evaluate on test set
criterion = nn.CrossEntropyLoss()
test_loss = 0.0
test_preds = []
test_labels = []

with torch.no_grad():
    for images, labels in tqdm(test_loader, desc="Testing"):
        images, labels = images.to(device), labels.to(device)
        outputs = best_model(images)
        loss = criterion(outputs, labels)
        
        test_loss += loss.item()
        _, preds = torch.max(outputs, 1)
        test_preds.extend(preds.cpu().numpy())
        test_labels.extend(labels.cpu().numpy())

test_loss /= len(test_loader)
test_acc = accuracy_score(test_labels, test_preds)
test_prec = precision_score(test_labels, test_preds, average='weighted', zero_division=0)
test_rec = recall_score(test_labels, test_preds, average='weighted', zero_division=0)
test_f1 = f1_score(test_labels, test_preds, average='weighted', zero_division=0)

print("\n" + "="*70)
print(f"TEST SET RESULTS - {best_model_name}")
print("="*70)
print(f"Loss:      {test_loss:.4f}")
print(f"Accuracy:  {test_acc:.4f} ({test_acc*100:.2f}%)")
print(f"Precision: {test_prec:.4f}")
print(f"Recall:    {test_rec:.4f}")
print(f"F1 Score:  {test_f1:.4f}")
print("="*70)

# Per-class metrics
print("\nPer-Class Performance:")
print("-" * 70)
report = classification_report(test_labels, test_preds, target_names=DISPLAY_NAMES, digits=4)
print(report)

## 11. Confusion Matrix

In [ ]:
# Confusion matrix
cm = confusion_matrix(test_labels, test_preds)
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

# Plot confusion matrix
fig, axes = plt.subplots(1, 2, figsize=(20, 8))

# Raw counts
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=DISPLAY_NAMES, yticklabels=DISPLAY_NAMES,
            cbar_kws={'label': 'Count'}, annot_kws={'size': 14})
axes[0].set_title(f'Confusion Matrix (Counts) - {best_model_name}',
                 fontsize=14, fontweight='bold')
axes[0].set_xlabel('Predicted', fontsize=12)
axes[0].set_ylabel('Actual', fontsize=12)

# Normalized
sns.heatmap(cm_normalized, annot=True, fmt='.3f', cmap='Blues', ax=axes[1],
            xticklabels=DISPLAY_NAMES, yticklabels=DISPLAY_NAMES,
            cbar_kws={'label': 'Proportion'}, annot_kws={'size': 14})
axes[1].set_title(f'Confusion Matrix (Normalized) - {best_model_name}',
                 fontsize=14, fontweight='bold')
axes[1].set_xlabel('Predicted', fontsize=12)
axes[1].set_ylabel('Actual', fontsize=12)

plt.tight_layout()
plt.savefig(os.path.join(output_dir, 'results', 'confusion_matrix.png'),
           dpi=150, bbox_inches='tight')
plt.show()

print("Confusion matrix saved!")

## 12. Visualize Predictions

In [ ]:
def visualize_predictions(model, dataloader, class_names, device, num_samples=12):
    """Visualize model predictions"""
    model.eval()
    
    fig, axes = plt.subplots(3, 4, figsize=(20, 15))
    axes = axes.flatten()
    
    data_iter = iter(dataloader)
    images, labels = next(data_iter)
    images_gpu = images.to(device)
    
    with torch.no_grad():
        outputs = model(images_gpu)
        probs = torch.softmax(outputs, dim=1)
        confidences, preds = torch.max(probs, 1)
    
    for i in range(min(num_samples, len(images))):
        # Denormalize
        img = images[i].permute(1, 2, 0).numpy()
        img = img * np.array([0.229, 0.224, 0.225]) + np.array([0.485, 0.456, 0.406])
        img = np.clip(img, 0, 1)
        
        true_label = labels[i].item()
        pred_label = preds[i].item()
        confidence = confidences[i].item()
        
        true_class = class_names[true_label]
        pred_class = class_names[pred_label]
        
        # Plot
        axes[i].imshow(img)
        
        # Color code: green for correct, red for incorrect
        color = 'green' if true_label == pred_label else 'red'
        
        title = f'True: {true_class}\nPred: {pred_class} ({confidence:.1%})'
        axes[i].set_title(title, fontsize=12, fontweight='bold', color=color)
        axes[i].axis('off')
    
    plt.suptitle(f'Predictions - {best_model_name}', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.savefig(os.path.join(output_dir, 'results', 'predictions.png'),
               dpi=150, bbox_inches='tight')
    plt.show()

print("Generating prediction visualizations...")
visualize_predictions(best_model, test_loader, DISPLAY_NAMES, device)

## 13. Save Results Summary

In [ ]:
# Generate comprehensive summary
results_text = f"""Microplastics Classification Method Comparison
================================================

Dataset: {DATASET_PATH}
Classes: {NUM_CLASSES} ({', '.join(DISPLAY_NAMES)})
Total images: {len(full_dataset)}
Train: {len(train_dataset)} | Val: {len(val_dataset)} | Test: {len(test_dataset)}

Training Configuration:
----------------------
Image size: 224x224
Batch size: {BATCH_SIZE}
Epochs: {NUM_EPOCHS}
Learning rate: {LEARNING_RATE}
Optimizer: Adam
Loss function: CrossEntropyLoss

Models Compared:
---------------
{comparison_df.to_string(index=False)}

Best Model: {best_model_name}
============================================

Test Set Performance:
--------------------
Accuracy:  {test_acc:.4f} ({test_acc*100:.2f}%)
Precision: {test_prec:.4f}
Recall:    {test_rec:.4f}
F1 Score:  {test_f1:.4f}
Loss:      {test_loss:.4f}

Per-Class Performance:
---------------------
{classification_report(test_labels, test_preds, target_names=DISPLAY_NAMES, digits=4)}

Confusion Matrix:
----------------
{cm}

Generated Files:
---------------
Models:
  - resnet50_best.pth
  - efficientnet_b0_best.pth
  - mobilenetv2_best.pth

Visualizations:
  - classification_comparison.csv
  - training_curves_comparison.png
  - confusion_matrix.png
  - predictions.png
  - sample_images.png
  - results.txt (this file)

Recommendation:
--------------
Best model for microplastic classification: {best_model_name}
Accuracy: {results[best_model_name]['best_val_acc']:.4f}

Model characteristics:
- ResNet50: Strong baseline, good accuracy, more parameters
- EfficientNet-B0: Best efficiency, balanced accuracy/speed
- MobileNetV2: Fastest inference, good for deployment

Choose based on your priorities:
- Highest accuracy → ResNet50 or EfficientNet-B0
- Fast inference/deployment → MobileNetV2
- Best balance → EfficientNet-B0
"""

# Save results
with open(os.path.join(output_dir, 'results', 'results.txt'), 'w') as f:
    f.write(results_text)

print("="*70)
print("ALL RESULTS SAVED")
print("="*70)
print(f"\nBest Model: {best_model_name}")
print(f"Test Accuracy: {test_acc*100:.2f}%")
print(f"\nFiles saved to: {os.path.join(output_dir, 'results')}")
print("="*70)

## Summary

### What This Notebook Does:

This notebook compares **3 deep learning classification methods** for identifying microplastic particle types (Bead, Fragment, Fiber). All models are trained on the same dataset splits with identical hyperparameters for fair comparison.

### Methods Compared:

1. **ResNet50** - Deep residual network (50 layers) with skip connections. Strong baseline, high accuracy.

2. **EfficientNet-B0** - Compound scaling architecture. Best efficiency with competitive accuracy.

3. **MobileNetV2** - Lightweight mobile architecture. Fast inference, good for deployment.

### Key Features:

- Transfer learning from ImageNet pre-trained weights
- Data augmentation (flips, rotations, brightness, noise)
- Cross-entropy loss with Adam optimizer
- Learning rate scheduling
- Comprehensive evaluation metrics (accuracy, precision, recall, F1)
- Confusion matrix analysis
- Visual predictions with confidence scores

### Typical Results:

- **Accuracy**: 85-95% depending on dataset quality
- **Best model**: Usually EfficientNet-B0 or ResNet50
- **MobileNetV2**: Slightly lower accuracy but 2-3x faster inference

### How to Use Results:

1. **Check comparison table** - Compare all models at a glance
2. **Review training curves** - Ensure no overfitting
3. **Analyze confusion matrix** - Identify which classes are confused
4. **Examine per-class metrics** - Find weak points
5. **Choose model** based on your priority (accuracy vs speed)

### For Your Presentation:

Include these visualizations:
- Comparison table showing all 3 methods
- Training curves (accuracy and F1 score)
- Confusion matrix of best model
- Sample predictions with confidence

### Next Steps:

1. Combine with segmentation for complete pipeline
2. Try ensemble methods (combine predictions)
3. Fine-tune best model with more epochs
4. Deploy best model to web app or mobile
5. Add more microplastic types if available

### Inference Example:

```python
# Load best model
model = all_models[best_model_name]
model.load_state_dict(torch.load('resnet50_best.pth'))
model.eval()

# Predict single image
pred_class, confidence, probs = predict_single_image(
    model, 'test_image.jpg', DISPLAY_NAMES, device
)
print(f"Predicted: {pred_class} ({confidence:.1%} confidence)")
```